# Using Python for Research: Final Project

## Introduction

In this final project, I'm tasked to predict the type of physical activity (e.g., walking, climbing stairs, sitting) from tri-axial smartphone accelerometer data. The dataset is split into 4 CSV files containing train_time_series, train_labels, test_time_series, test_labels respectively. The time series data is presented in the following format:
`timestamp, UTC time, accuracy, x, y, z`
and the labels are provided as:
`timestamp, label`. However the test_labels don't contain the labels and the task is to predict and fill them.

### Summary of steps performed in this task

 - First I read the files to pandas dataframes and then cleaned up the data.
 - I studied the data using box plot and pair plot from Seaborn. 
 - After looking at the plots, I decided to use rolling mean and standard deviation of the time series data as covariates along with the x, y and z variates. 
 - I used Random Forest Regression from `scikit-learn` to solve the task at hand. 
 - I studied about various neural network architectures are used to solve time series classification problems, but still felt that a simple Random Forest Classifier would be sufficient to gain good accuracy. 
 - The test labels are then predicted using the now fitted `forest_classifier` and stored in a CSV file. 
 - An additional code block is given at the end for runtime calculation.

## Methods

### Step 1

First, we will import some libraries that will help in solving the task at hand. 
 - `scikit-learn` (**sklearn**) contains helpful statistical models, in particular `RandomForestClassifier` that we'll use for classification and also some additional helper functions like `train_test_split`, `cross_val_score`, `confusion_matrix`, `classification_report`
 - `matplotlib.pyplot` and `seaborn` libraries are used here for data and metric visualizations.
 - `numpy` and `pandas` for data manipulation (obviously).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, classification_report
import seaborn as sns
sns.set()
pd.options.mode.chained_assignment = None

### Step 2
Using `pd.read_csv`, the CSV files are read into dataframes of the same name. `index_col=0` is used to set the first column in csv as index.

In [ ]:
train_time_series = pd.read_csv("train_time_series.csv", index_col=0)
train_labels = pd.read_csv("train_labels.csv", index_col=0)
test_time_series = pd.read_csv("test_time_series.csv", index_col=0)
test_labels = pd.read_csv("test_labels.csv", index_col=0)

In [ ]:
print("Length of time series data: %d and Length of labelled data: %d" % (len(train_time_series), len(train_labels)))

In [ ]:
train_time_series

In [ ]:
train_labels

### Step 3

As seen above the raw data and actual labelled data are not equal. This is because the labels are only provided for every 10th observation. We can just extract the raw data to match the labels,but then the data would be quite low. Since we know that the labels don't change suddenly, it is possible to augment the data with preceding values. It is not perfect, but it is better than dealing with very low data.

So we reindex the train labels with `reindex` method using new indices generated by `range` function.

In [ ]:
new_index = list(range(train_labels.index[0], train_labels.index[-1] + 1))
train_labels = train_labels.reindex(new_index)
train_labels

### Step 4 

Before filling the NaN values in train labels, we'll deal with the time series data first. Since the data is time series data, it is better if we consider not only the current value but also the influence of past values. A simple way to approach this would be to implement a **sliding window** (also known as **rolling window**).

![sliding window](https://www.splunk.com/content/dam/splunk-blogs/signalfx-assets/blog-images//incremental-decremental.png)

We will make use of the `rolling` method to find the *mean* and *standard deviation* of rolling window of size **100**. The window size is just a heuristic and was chosen by trial-and-error basis. The first value of `std` columns would be NaN, so `fillna` method is used with `method=bfill`

In [ ]:
train_time_series[["std_x", "std_y", "std_z"]] = train_time_series[["x", "y", "z"]].rolling(window=100, min_periods= 1).std()
train_time_series[["mean_x", "mean_y", "mean_z"]] = train_time_series[["x", "y", "z"]].rolling(window=100, min_periods =1).mean()
train_time_series.fillna(method="bfill", inplace=True)
train_time_series

### Step 4 

Now that we got our required covariates, it is time to create our `X` and `y` dataframes which will be the arguments for the Random Forest Classifier. We also fill the NaN labels in y using the same `fillna` method seen above, but with `method=ffill` indicating forward fill

In [ ]:
covariates = ["x","y","z","std_x", "std_y", "std_z", "mean_x", "mean_y", "mean_z"]

# Note that time_series data starts at index 20586 but labels start at 20589
# We will splice out the covariates only from 20589 by leaving out first 3 indices
X = train_time_series[covariates][3:]
y = train_labels["label"]
y.fillna(method="ffill", inplace=True)

In [ ]:
X

In [ ]:
y

In [ ]:
# Skew of covariates
X.skew()

### Step 5

For data visualization, we will make use of the seaborn library. First we'll concatenate our `X` and `y` dataframes to create `Xy`. Then we will create a 3x3 plotting grid for visualizing the data distributuon of our 9 covariates. Finally we make use of `boxenplot` function to plot enhanced box plots. The plot provides more information about the nature of distribution of data.

In [ ]:
# We'll use pd.concat function with axis=1 for column-wise concatenation
Xy = pd.concat([X,y],axis=1) 
fig, axes = plt.subplots(3,3, figsize=(15,15))
for index,cov in enumerate(covariates):
    i,j = index // 3, index % 3; # for getting the resepctive axes
    sns.boxenplot(x="label", y=cov, data=Xy, ax=axes[i][j])
fig.suptitle("Boxplot of covariates");

From the above plots we can see that the x, y and z boxplots have almost the same IQR for every class and follow fairly standard normal distibution. But looking at the rolling mean and standard deviation boxplots, we see greater variability in the median line and IQR, thus we can use these to achieve high accuracy

### Step 6

Now we'll plot the marginal distribution of the univariates and pairwise scatter plot of the covariates using `pairplot` function.

In [ ]:
sns.pairplot(Xy[["x","y","z","label"]], hue="label", palette = "hls");

In [ ]:
sns.pairplot(Xy[["std_x","std_y","std_z","label"]], hue="label", palette = "hls");

In [ ]:
sns.pairplot(Xy[["mean_x","mean_y","mean_z","label"]], hue="label", palette = "hls");

By looking at the pairplot of x, y and z, we can infer that data has low bias and high variance. The standard deviation pairplot has noticable right skew, indicating positive correlation. The mean pairplot has slight skew, and some clusters can be seen.

### Step 7

Finally we'll plot the correlation matrix using `heatmap` function

In [ ]:
sns.heatmap(Xy.corr());

The correlation matrix further shows that the rolling standard deviation has strong positive correlation and also the rolling mean is also correlated with labels

### Step 8

Now it is time to train the classifier. But first we'll split our training data into `train` and `validation` sets with 80/20 split. We'll make use of scikit-learn's `train_test_split` function for this

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2)

In [ ]:
print("Length of training split: %d\nLength of validation split: %d" % (len(X_train), len(X_validation)))

### Step 9

We'll create an instance of the classifier using `RandomForestClassifier()`. To find the best hyperparameters for our classifier, we'll use the `RandomizedSearchCV` function from scikit-learn. This will help us estimate the `n_estimators` and `max_depth`. Then we'll use the parameters to fit our data 

Note that the default arguments for `RandomForestClassifier()` work pretty well and hence this hyperparameter serach is not necessary, but it's good practice to search for optimal parameters

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# We'll search 10 n_estimators from 200 to 2000 and 12 max_depth parameters from 10 to 120 (along with default `None`)
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)]
max_depth.append(None)

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth}

In [ ]:
rf = RandomForestClassifier()

# We'll search for 100 candidates with 3 fold cross validation
# This is a time consuming process, so setting n_jobs = -1 will make use of all the available cores.

forest_classifier = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, n_jobs = -1)
forest_classifier.fit(X_train, y_train)

In [ ]:
forest_classifier.best_params_

This shows the best hyperparameters obtained after `RandomizedSearchCV`. We'll use these best parameters directly next time (while calculating runtime). Now that the fitting is done, it is time to evaluate our model

## Results

We'll now score our classifier with the validation set using the `score` method

In [ ]:
forest_classifier.score(X_validation, y_validation)

Time to get the classification report and the confusion matrix plot using functions of the same name

In [ ]:
# classification_report function expects prediction output as argument, so we use `predict` method first

y_validation_predicted = forest_classifier.predict(X_validation)
class_report = classification_report(y_validation_predicted, y_validation)
print(class_report)

In [ ]:
plot_confusion_matrix(forest_classifier, X_validation, y_validation);

The classifier has high accuracy, great F1 score and the confusion matrix looks good with very little misclassified labels.

Now we'll predict the labels for the test dataset provided. We add the rolling mean and standard deviation columns similar to what we did for train_time_series, and then predict the labels

In [ ]:
testX = test_time_series[["x","y","z"]]
testX[["std_x", "std_y", "std_z"]] = test_time_series[["x","y","z"]].rolling(window=100, min_periods=1).std().fillna(method="bfill")
testX[["mean_x", "mean_y", "mean_z"]] = test_time_series[["x","y","z"]].rolling(window=100, min_periods=1).mean()

In [ ]:
testX

In [ ]:
testY = forest_classifier.predict(testX)

In [ ]:
testY

Similar to the train_labels, test_labels are only required for every 10th observation. So we splice the indices as required and store them in the `label` column. This could also be done by taking the mean/mode of every 10 predictions. We cast them as type `int` as that is how it was in the original train_labels

In [ ]:
test_labels["label"] = testY[9::10].astype(int)

Finally, we export the test_labels dataframe as *test_labels_filled.csv* using pandas' `to_csv` method

In [ ]:
test_labels.to_csv("test_labels_filled.csv")

## Summary

We have seen how a Random Forest Classifier can be used to classify time series data. 
 - First we imported the data as pandas dataframes, then we filled in the missing gaps in labels using  `ffill`.
 - We used rolling window method to make use of the time-dependent nature of data, and created 6 additional covariates `std` and `mean` for each x, y and z. 
 - We studied these covariates using `boxenplot`, `pairplot` and a correlation heatmap. These plots displayed the high positive correlation in the rolling standard_deviation and also the correlation between the rolling mean and labels.
 - We split our dataset into train and validation (80/20 split)
 - We used RandomizedSearchCV to estimate the best values for the hyperparameters `n_estimators` and `max_depth`
 - We then created a random forest classifer instance to fit our training data.
 - We evaluated our classifier using our validation data and obtained our classification report and confusion matrix, all of which showed great accuracy and F1 score.
 - Finally we used the classifier to predict the labels of given test_time_series data and exported the predictions to a CSV file

## Runtime Calculation

using the `%%time` magic command from IPython, we'll calculate the runtime of the code given, which excludes data visualization, metrics, hyperparameter search etc.

In [ ]:
%%time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, classification_report
import seaborn as sns
sns.set()
pd.options.mode.chained_assignment = None

train_time_series = pd.read_csv("train_time_series.csv", index_col=0)
train_labels = pd.read_csv("train_labels.csv", index_col=0)
test_time_series = pd.read_csv("test_time_series.csv", index_col=0)
test_labels = pd.read_csv("test_labels.csv", index_col=0)

new_index = list(range(train_labels.index[0], train_labels.index[-1] + 1))
train_labels = train_labels.reindex(new_index)

train_time_series[["std_x", "std_y", "std_z"]] = train_time_series[["x", "y", "z"]].rolling(window=100, min_periods= 1).std()
train_time_series[["mean_x", "mean_y", "mean_z"]] = train_time_series[["x", "y", "z"]].rolling(window=100, min_periods =1).mean()
train_time_series.fillna(method="bfill", inplace=True)

covariates = ["x","y","z","std_x", "std_y", "std_z", "mean_x", "mean_y", "mean_z"]
X = train_time_series[covariates][3:]
y = train_labels["label"]
y.fillna(method="ffill", inplace=True)

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2)

forest_classifier = RandomForestClassifier(max_depth=None, n_estimators=800)
forest_classifier.fit(X_train, y_train)
print(forest_classifier.score(X_validation, y_validation))

testX = test_time_series[["x","y","z"]]
testX[["std_x", "std_y", "std_z"]] = test_time_series[["x","y","z"]].rolling(window=100, min_periods=1).std().fillna(method="bfill")
testX[["mean_x", "mean_y", "mean_z"]] = test_time_series[["x","y","z"]].rolling(window=100, min_periods=1).mean()

testY = forest_classifier.predict(testX)
test_labels["label"] = testY[9::10].astype(int)
test_labels.to_csv("test_labels_filled.csv")